### CaseI: Midwest MCS Analysis and Mesoscale Perturbation Tracking 

This notebook provides visualizations of surface analysis during a long-lasting MCS event over the upper-midwest.

1. Import relevant Python libraries and setup cartopy/colortables

2. Plot hybrid objective analysis displaying MADIS observations overlayed with pressure contours from smartphone analyes and temperature/dew point contours from MADIS anlayses

3. Plot gridded surface analysis, for CaseI, using smartphone pressure analyses and MADIS analyses of temperature, dew point, and wind.

4. Plot mesoscale surface perturbation analyses, for Case II, derived from band-passed filtered smartphone pressure analyses and MADIS analyses of temperature, dew point, and wind.

In [1]:
### ---- (1) ---- ####
#Import Python libraries

import os
import sys
sys.path.append('../../PyScripts')
import xarray as xr
import matplotlib
import cmasher as cmr
from matplotlib import pyplot as plt
import numpy as np
import funcs
import sys
import lcmaps
import colorcet as cc
import cmasher as cmr
from datetime import datetime,timedelta
from cartopy.feature import NaturalEarthFeature,BORDERS,LAKES,COLORS
import cartopy.crs as crs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from metpy.plots import colortables
from scipy import ndimage
from scipy import signal
import pandas as pd
from scipy.signal import butter, lfilter
import multiprocessing
from joblib import Parallel,delayed

from metpy.calc import reduce_point_density
from metpy.plots import StationPlot

#Retrieve perceptually uniform colorbar from colorcet
cmapp = cc.cm.rainbow_bgyrm_35_85_c71

#Set format for datetime objects
fmt = '%Y%m%d_%H%M'

# Download/add state and coastline features for cartopy 
states = NaturalEarthFeature(category="cultural", scale="10m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")

land_50m = NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='k',
                                        facecolor='none')

#Define function to add map data to matplotlib plot
def add_map(ax,clr,lw):
    ax.add_feature(states)
    ax.add_feature(BORDERS)
    ax.add_feature(land_50m)
    ax.add_feature(states,edgecolor=clr,lw=lw)
    ax.add_feature(LAKES, edgecolor=clr)

#Define function to add latitude/longitude grid lines to cartopy/matplotlib plot
def add_gridlines(ax,xl,yl,clr, fs):
    gl = ax.gridlines(crs=crs.PlateCarree(), draw_labels=True,
                      linewidth=0.25, color=clr, alpha=1, linestyle='--')

    gl.xlabels_bottom = xl
    gl.xlabels_top = False
    gl.ylabels_left = yl
    gl.ylabels_right = False

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': fs, 'color': clr}
    gl.ylabel_style = {'size': fs, 'color': clr}
    return gl

#Get Composite Reflectivity colormap from metpy

ctable0 = 'Carbone42'
ctable1 = 'NWSStormClearReflectivity'
ctable2 = 'ir_rgbv'

cmapp = cc.cm.rainbow_bgyrm_35_85_c71

norm1, cmapp_prs = colortables.get_with_steps(ctable0, 140, 140)
norm, cmapp_radar = colortables.get_with_steps(ctable1, 244, 244)
norm2, cmapp_sat = colortables.get_with_steps(ctable2, 140, 140)

#Increase with of notebook to fill screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#Define function to mask pressure analyses over water
def mask_grid(arr):
    arr = np.ma.masked_where(landsea==0,arr)
    return arr

#Define function to read and subset a land/sea boolean grid
def get_landsea():
    ds_land = xr.open_dataset('../../../data/Static/landsea.nc')
    ds_land = funcs.subset(ds_land,minLat,maxLat,minLng,maxLng)
    landsea = ds_land['LANDSEA'].values
    landsea = np.pad(landsea, ((0,1),(0,1)), 'edge')
    ds_land.close()
    return landsea

#Get bounds of feature track
def get_bounds(lats,lngs):

    clat = np.mean(lats)
    clng = np.mean(lngs)

    dy = round((lats.max()-lats.min())+1)
    dx = round((lngs.max()-lngs.min())+1)

    if (dy < 2.5):
        dy = 2.5

    dx = dy*1.75

    #if (dy < 2.5):

    minLat = clat-dy
    maxLat = clat+dy
    minLng = clng-dx
    maxLng = clng+dx

    bounds = [minLat,maxLat,minLng,maxLng]
    return bounds

In [11]:
### ---- (2) ---- ####
#Plot hybrid objective analysis displaying MADIS observations overlayed with pressure contours from smartphone analyes and temperature/dew point contours from MADIS anlayses

#Retrieve surface analyses variables for CaseI
dg1 = xr.open_dataset('../../../data/Cases/caseI_anal.nc')
alts = dg1['altimeter'].values
temp = dg1['temperature'].values
dewp = dg1['dewpoint'].values
uwind = dg1['uwind'].values
vwind = dg1['vwind'].values

#Retrieve composite radar data for CaseI
dg2 = xr.open_dataset('../../../data/Cases/caseI_cref.nc')
refl = dg2['REFL'].values
dg2.close()

#Retrieve list of dates from surface analysis dataset
dtlist0 = list(pd.to_datetime(dg1['Valid'].values))
dtlist1 = [datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S') for d in dtlist0]
dtlist = [datetime.strftime(d,'%Y%m%d_%H%M') for d in dtlist1]

#Retrieve latitude/longitude from dataset
ygrid = dg1['longitude'].values; xgrid = dg1['latitude'].values
X,Y = np.meshgrid(ygrid,xgrid) #Create 2d coordinates for contour plotting

#Get contour levels for altimeter 
alt_min = np.round(np.nanmin(alts)-1,0)
alt_max = np.round(np.nanmax(alts)+1,0)

#Handle cases where range of pressure values are large (e.g. hurricanes)
if (((alt_max-alt_min) > 10) and ((alt_max-alt_min) < 20)):
    da = 2
elif ((alt_max-alt_min) > 20):
    da = 5
else:
    da = 1
    
#Round pressure levels for contouring
aran = np.arange(alt_min,alt_max,da)
aran = da*np.round(aran/da)

#Get contour levels for temperature 
tmp_min = np.round(temp.min()-1,0)
tmp_max = np.round(temp.max()+1,0)
tran = np.arange(tmp_min,tmp_max,2)

#Get contour levels for dew point temperature 
dpt_min = np.round(dewp.min()-1,0)
dpt_max = np.round(dewp.max()+1,0)
dran = np.arange(dpt_min,dpt_max,2)

#Retrieve feature track for CaseI
dst = xr.open_dataset('../../../data/Cases/caseI_path.nc')
tlats = dst['lats'].values
tlngs = dst['lngs'].values
tims = dst['time'].values
dst.close()

#Get bounds of track for figure axis limits
bounds = get_bounds(tlats,tlngs)
minLat,maxLat,minLng,maxLng = bounds
frm = np.arange(0,len(tims))

#Determine scale at which to filter point observations from stations (reduce density for visibility)
dxx = maxLng-minLng
dyy = maxLat-minLat
dxs = max([dxx,dyy])
dxy = dxs/25.0

#List of variables
vvars = ['altimeter','temperature','dewpoint','wind']
dobs = []

#Read in MADIS point observation data from netcdf
for v in range(0,len(vvars)):
    obs_ds = xr.open_dataset('../../../data/Cases/observations/caseI_'+str(vvars[v])+'_obs.nc')
    dobs.append(obs_ds)
    obs_ds.close()

#Set base font size
matplotlib.rcParams.update({'font.size': 20})
def plot_obj(d,ddate):

    #Set figure size
    fig = plt.figure(figsize=(20,10))

    #Get frame index
    fi = np.argwhere(frm<=d).T[0]
    fl = fi[-1]
    
    #Add cartopy map to figure 
    ax1 = plt.subplot(111,projection=crs.PlateCarree())
    add_map(ax1,'dimgray',1)
    add_gridlines(ax1,True,True,'k',20)

    #Loop through list of observation datasets (one dataset per variable)
    for j in range(0,len(dobs)):
            #Find observations valid during the current frame (i.e. 5-min time window)
            dob = dobs[j]
            dob = dob.where(dob.valid==tims[fl],drop=True)
            #Subset dataset to plotting region (bounding box)
            data = funcs.subset(dob,minLat,maxLat,minLng,maxLng)
            data = data.to_dataframe()
            #Get location of point observations
            point_locs = np.array([data['longitude'].values,data['latitude'].values]).T
            #Filter point obs to ensure visibility of station plots
            data = data[reduce_point_density(point_locs, dxy)]

            #Mark location of point observations
            ax1.scatter(data['longitude'].values,data['latitude'].values,c='k',s=5,zorder=5)
            #Add station plot
            stationplot = StationPlot(ax1, data['longitude'].values, data['latitude'].values,clip_on=True, transform=crs.PlateCarree(), fontsize=12, zorder=5)
            if (j == 0):
                    stationplot.plot_parameter('NE', data['altimeter'].values,formatter=lambda v: format(10 * v, '.0f')[-3:],zorder=5)
            if (j == 1):
                    stationplot.plot_parameter('NW', data['tmpf'].values, color='red',zorder=5)
            if (j == 2):
                    stationplot.plot_parameter('SW', data['dwpf'].values,color='darkgreen',zorder=5)
            if (j == 3):
                    stationplot.plot_barb(data['uwind'].values, data['vwind'].values,zorder=5)

    #Contours from smartphone pressure (RTS) analyses
    llist = []
    alt_smooth = ndimage.gaussian_filter(alts[d], sigma=5, order=0)
    cs1 = ax1.contour(X,Y,alt_smooth,levels=aran,colors='k',zorder=4)
    llist.append('P')
    ax1.clabel(cs1,cs1.levels,fmt="%1.0f",inline=True,fontsize=12)

    #Contours from MADIS temperature (RTS) analyses
    tmp_smooth = ndimage.gaussian_filter(temp[d], sigma=5, order=0)
    cs2 = ax1.contour(X,Y,tmp_smooth,levels=tran,colors='firebrick',linestyles='dashed',zorder=4)
    llist.append('T')
    ax1.clabel(cs2,cs2.levels,fmt="%1.0f",inline=True,fontsize=12)
    
    #Contours from MADIS dewpoint temperature (RTS) analysis
    dpt_smooth = ndimage.gaussian_filter(dewp[d], sigma=5, order=0)
    cs3 = ax1.contour(X,Y,dpt_smooth,levels=dran,colors='forestgreen',linestyles='dotted',zorder=4)
    llist.append('Td')
    ax1.clabel(cs3,cs3.levels,fmt="%1.0f",inline=True,fontsize=12)

    #Add composite reflevity to map (shaded)
    im4 = ax1.imshow(refl[d],origin='lower',aspect='auto',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=cmapp_radar,vmin=-32,vmax=90.,alpha=0.6)
    #ax1.streamplot(X,Y,uwind[d],vwind[d],transform=crs.PlateCarree(),color='k')
    #llist.append('Streamlines (U,V)')
    cb4=plt.colorbar(im4)
    llist.append('CREF')
    cb4.ax.set_title('(dBZ)')
    
    #Plot path of perturbation track (note this may not align with the absolute pressure! Features follow perturbations (see next figure) not local minima/maxima in pressure)
    ax1.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax1.set_ylim([bounds[0],bounds[1]])
    ax1.set_xlim([bounds[2],bounds[3]])
    
    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
    
    #Add title and save to file
    fig.canvas.draw()
    plt.tight_layout()
    
    plt.subplots_adjust(top=0.93)
    plt.title("Surface Objective Analyses: "+", ".join(llist)+" "+ddate[9:15]+" UTC "+ddate[4:6]+"/"+ddate[6:8]+"/"+ddate[0:4],fontsize=24)
    plt.savefig('../../../Plots/Cases/kfpoint_caseI_'+dd+'.png')
    ax1.cla()    
    plt.close()
    
#Perform plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_obj)(d,ddate) for d,ddate in enumerate(dtlist))

In [12]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../../../Plots/Cases/kfpoint_caseI.mp4'):
    os.system('rm -rf ../../../Plots/Cases/kfpoint_caseI.mp4')
#Create mp4 movie from 5-min surface objective analysis during CaseI
os.system('ffmpeg -r 6 -f image2 -s 1920x1080 -i ../../../Plots/Cases/kfpoint_caseI_%03d.png -c:v libx264 -pix_fmt yuv420p ../../../Plots/Cases/kfpoint_caseI.mp4')
#(Below) display video

0

In [13]:
%%HTML
<div align="middle">
<video width="100%" controls>
      <source src = "../../../Plots/Cases/kfpoint_caseI.mp4" type="video/mp4">
</video></div>

In [14]:
### ---- (3) ---- ####
#Define function to plot gridded surface analysis, for CaseI, using smartphone pressure analyses and MADIS analyses of temperature, dew point, and wind.
def plot_pert(d,ddate):
         
    #Set figure size
    fig = plt.figure(figsize=(16,8))
    ax1 = plt.subplot(221,projection=crs.PlateCarree())
    #Add cartopy map to figure
    add_map(ax1,'dimgray',1)
    add_gridlines(ax1,False,True,'k',14)

    #Plot smartphone pressure analysis
    #Smooth altimeter field for contouring
    alts_smooth = ndimage.gaussian_filter(alts[d], sigma=2, order=0)
    im1 = ax1.imshow(alts[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=cmapp,vmin=alt_min,vmax=alt_max)
    #Add and label isobars
    cs11 = ax1.contour(X,Y,alts_smooth,levels=aran,colors='k')
    ax1.clabel(cs11,cs11.levels,fmt="%1.0f",inline=True,fontsize=9)
    cb=plt.colorbar(im1)
    cb.ax.tick_params(labelsize=14)
    ax1.set_title('Altimeter (hPa)',fontsize=16)

    #Plot MADIS temperature analysis
    ax2 = plt.subplot(222,projection=crs.PlateCarree())
    add_map(ax2,'dimgray',1)
    add_gridlines(ax2,False,False,'k',14)
    im2 = ax2.imshow(temp[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=lcmaps.thetae(),vmin=tmp_min,vmax=tmp_max)
    #Add streamlines from MADIS wind analysis
    ax2.streamplot(X, Y, uwind[d], vwind[d], transform=crs.PlateCarree(), color='k')
    ax2.set_title("Temperature (deg F)",fontsize=16)
    cb=plt.colorbar(im2)
    cb.ax.tick_params(labelsize=14)

    #Plot MADIS dew point temperature analysis
    ax3 = plt.subplot(223,projection=crs.PlateCarree())
    add_map(ax3,'dimgray',1)
    add_gridlines(ax3,True,True,'k',14)
    im3 = ax3.imshow(dewp[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=cmapp_sat,vmin=dpt_min,vmax=dpt_max)
    #Add streamlines from MADIS wind analysis
    ax3.streamplot(X, Y, uwind[d], vwind[d], transform=crs.PlateCarree(), color='k')
    ax3.grid(True)
    ax3.set_title("Dew Point (deg F)",fontsize=16)
    cb=plt.colorbar(im3)
    cb.ax.tick_params(labelsize=14)

    #Plot composite reflectivity analysis
    ax4 = plt.subplot(224,projection=crs.PlateCarree())
    add_map(ax4,'dimgray',1)
    add_gridlines(ax4,True,False,'k',14)
    im4 = ax4.imshow(refl[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=cmapp_radar,vmin=-32,vmax=90)
    #Overlay isobars from smarpthone pressure analyses
    cs11 = ax4.contour(X,Y,alts_smooth,levels=aran,colors='k')
    ax4.clabel(cs11,cs11.levels,fmt="%1.0f",inline=True,fontsize=9)
    ax4.set_title("CREF (dBz)",fontsize=16)
    cb=plt.colorbar(im4)
    cb.ax.tick_params(labelsize=14)

    #Get index of past frames
    fi = np.argwhere(frm<=d).T[0]

    #Plot feature track in all subplots
    ax1.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax1.set_ylim([bounds[0],bounds[1]])
    ax1.set_xlim([bounds[2],bounds[3]])

    ax2.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax2.set_ylim([bounds[0],bounds[1]])
    ax2.set_xlim([bounds[2],bounds[3]])

    ax3.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax3.set_ylim([bounds[0],bounds[1]])
    ax3.set_xlim([bounds[2],bounds[3]])

    ax4.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax4.set_ylim([bounds[0],bounds[1]])
    ax4.set_xlim([bounds[2],bounds[3]])

    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
    
    fig.canvas.draw()
    plt.tight_layout()
    #Add title and save figure
    plt.subplots_adjust(top=0.9)
    plt.suptitle('5-min Surface Analysis '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    plt.savefig('../../../Plots/Cases/kfanal_caseI_'+dd+'.png')
    plt.close()
    
#Perform plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_pert)(d,ddate) for d,ddate in enumerate(dtlist))

In [15]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../../../Plots/Cases/kfanal_caseI.mp4'):
    os.system('rm -rf ../../../Plots/Cases/kfanal_caseI.mp4')
#Create mp4 movie from 5-min gridded surface analysis of CaseI
os.system('ffmpeg -r 6 -f image2 -s 1920x1080 -i ../../../Plots/Cases/kfanal_caseI_%03d.png -c:v libx264 -pix_fmt yuv420p ../../../Plots/Cases/kfanal_caseI.mp4')
#(Below) display video

0

In [16]:
%%HTML
<div align="middle">
<video width="100%" controls>
      <source src = "../../../Plots/Cases/kfanal_caseI.mp4" type="video/mp4">
</video></div>

In [20]:
### ---- (4) ---- ####
#Define function to plot mesoscale surface perturbation analyses, for Case I, derived from band-passed filtered smartphone pressure analyses and MADIS analyses of temperature, dew point, and wind.

#Retrieve surface perturbation analyses variables for CaseI
dg1 = xr.open_dataset('../../../data/Cases/caseI_pert.nc')
alts_meso = dg1['altimeter_meso'].values
temp_meso = dg1['temperature_meso'].values
dewp_meso = dg1['dewpoint_meso'].values
uwind_meso = dg1['uwind_meso'].values
vwind_meso = dg1['vwind_meso'].values

#Retrieve pre-generated Kalman smoothed LatticeKrig wind analysis for 14 April, 2018.
dg2 = xr.open_dataset('../../../data/Cases/caseI_cref.nc')
refl = dg2['REFL'].values
dg2.close()

#Retrieve list of dates from surface analysis dataset
dtlist0 = list(pd.to_datetime(dg1['Valid'].values))
dtlist1 = [datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S') for d in dtlist0]
dtlist = [datetime.strftime(d,'%Y%m%d_%H%M') for d in dtlist1]

#Retrieve latitude/longitude from dataset
ygrid = dg1['longitude'].values; xgrid = dg1['latitude'].values
X,Y = np.meshgrid(ygrid,xgrid) #Create 2d coordinates for contour plotting

#Set contour levels for smartphone pressure perturbations
aran = [-1.5,-1.25,-1,-0.75,0.75,1,1.25,1.5] 

#Get contour levels for MADIS temperature perturbations
tmp_min = np.round(np.nanmin(temp_meso)-1,0)
tmp_max = np.round(np.nanmax(temp_meso)+1,0)
if (abs(tmp_min)>tmp_max):
        tmp_max = -1*tmp_min
else:
        tmp_min = -1*tmp_max

tran = list(np.arange(tmp_min,tmp_max,0.5))
tran.remove(0)

#Get contour levels for MADIS dew point temperature perturbations
dpt_min = np.round(np.nanmin(dewp_meso)-1,0)
dpt_max = np.round(np.nanmax(dewp_meso)+1,0)
if (abs(dpt_min)>dpt_max):
        dpt_max = -1*dpt_min
else:
        dpt_min = -1*dpt_max

dran = list(np.arange(dpt_min,dpt_max,0.5))
dran.remove(0)

#Set minimum font size for plotting
matplotlib.rcParams.update({'font.size': 16})

### ---- (3) ---- ####
#Define function to plot gridded surface perturbation analysis, for CaseI, using band-pass filtered smartphone pressure analyses and MADIS analyses of temperature, dew point, and wind.
def plot_pert(d,ddate):
    
    #Set figure size
    fig = plt.figure(figsize=(16,8))
    ax1 = plt.subplot(221,projection=crs.PlateCarree())
    #Add cartopy map to figure
    add_map(ax1,'dimgray',1)
    add_gridlines(ax1,False,True,'k',14)

    #Plot mesoscale pressure perturbations derived from smartphone analyses
    im1 = ax1.imshow(alts_meso[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=plt.cm.PuOr_r,vmin=-1.5,vmax=1.5)
    im11 = ax1.contour(X,Y,alts_meso[d],levels=aran,colors='k')
    q1 = ax1.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)
    cb=plt.colorbar(im1)
    cb.ax.tick_params(labelsize=14)
    ax1.set_title("Altimeter (hPa)",fontsize=16)

    #Plot mesoscale temperature perturbations derived from MADIS analyses
    ax2 = plt.subplot(222,projection=crs.PlateCarree())
    add_map(ax2,'dimgray',1)
    add_gridlines(ax2,False,False,'k',14)

    im2 = ax2.imshow(temp_meso[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=plt.cm.RdBu_r,vmin=-1.5,vmax=1.5)
    im22 = ax2.contour(X,Y,temp_meso[d],levels=tran,colors='k')
    q2 = ax2.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)

    ax2.set_title("Temperature (deg C)",fontsize=16)
    cb=plt.colorbar(im2)
    cb.ax.tick_params(labelsize=14)

    #Plot mesoscale dew poinb temperature perturbations derived from MADIS analyses
    ax3 = plt.subplot(223,projection=crs.PlateCarree())
    add_map(ax3,'dimgray',1)
    add_gridlines(ax3,True,True,'k',14)

    im3 = ax3.imshow(dewp_meso[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=plt.cm.BrBG,vmin=-1.5,vmax=1.5)
    im33 = ax3.contour(X,Y,dewp_meso[d],levels=dran,colors='k')
    q3 = ax3.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)

    ax3.set_title("Dew Point (deg C)",fontsize=16)
    cb=plt.colorbar(im3)
    cb.ax.tick_params(labelsize=14)

    #Plot composite reflectivity analyses
    ax4 = plt.subplot(224,projection=crs.PlateCarree())
    add_map(ax4,'dimgray',1)
    add_gridlines(ax4,True,False,'k',14)

    im4 = ax4.imshow(refl[d],origin='lower',aspect='auto',interpolation='bilinear',extent=[X.min(),X.max(),Y.min(),Y.max()],cmap=cmapp_radar,vmin=-32,vmax=90)
    cs11 = ax4.contour(X,Y,alts_meso[d],levels=aran,colors='k')
    q4 = ax4.quiver(X[::5,::5],Y[::5,::5],uwind_meso[d][::5,::5],vwind_meso[d][::5,::5],scale=35,width=0.0025,zorder=10)

    ax4.clabel(cs11,cs11.levels,fmt="%1.2f",inline=True,fontsize=9)
    ax4.set_title("CREF (dBz)",fontsize=16)
    cb=plt.colorbar(im4)
    cb.ax.tick_params(labelsize=14)

    #Get index of past frames
    fi = np.argwhere(frm<=d).T[0]

    #Overlay feature track in each subplot
    ax1.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax1.set_ylim([bounds[0],bounds[1]])
    ax1.set_xlim([bounds[2],bounds[3]])

    ax2.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax2.set_ylim([bounds[0],bounds[1]])
    ax2.set_xlim([bounds[2],bounds[3]])

    ax3.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax3.set_ylim([bounds[0],bounds[1]])
    ax3.set_xlim([bounds[2],bounds[3]])

    ax4.plot(tlngs[fi],tlats[fi],'-ok',color='k',lw=4,zorder=7)
    ax4.set_ylim([bounds[0],bounds[1]])
    ax4.set_xlim([bounds[2],bounds[3]])
    
    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
    
    #Add title and save figure
    plt.subplots_adjust(top=0.93)
    plt.suptitle('5-min Perturbation Analysis '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    fig.canvas.draw()
    plt.tight_layout()
    plt.savefig('../../../Plots/Cases/kfpert_caseI_'+dd+'.png')
    plt.close()
    
#Perform plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_pert)(d,ddate) for d,ddate in enumerate(dtlist))

In [21]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../../../Plots/Cases/kfpert_caseI.mp4'):
    os.system('rm -rf ../../../Plots/Cases/kfpert_caseI.mp4')
#Create mp4 movie from surface perturbation analyses anlayses for Case I
os.system('ffmpeg -r 6 -f image2 -s 1920x1080 -i ../../../Plots/Cases/kfpert_caseI_%03d.png -c:v libx264 -pix_fmt yuv420p ../../../Plots/Cases/kfpert_caseI.mp4')
#(Below) display video

0

In [22]:
%%HTML
<div align="middle">
<video width="100%" controls>
      <source src = "../../../Plots/Cases/kfpert_caseI.mp4" type="video/mp4">
</video></div>